# TIF360 Project

Main source: https://www.kaggle.com/code/rmonge/predicting-molecule-properties-based-on-its-smiles/notebook

### Import packages

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import time as time
import torch
import torch_geometric

from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch.nn import Linear, LeakyReLU
from torch_geometric.nn import global_mean_pool, GATConv, BatchNorm, GraphNorm
import torch.nn.functional as F

from sklearn.metrics import r2_score
import utility_functions as uf  # defined in utility_functions.py
import GNN_structures as GNNs # defined in GNN_structures.py
import warnings
warnings.filterwarnings(action="once") # only displays the warning once

In [2]:
# check if cuda is available
print('cuda available:', torch.cuda.is_available())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', "cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print('cuda version:', torch.version.cuda)
    print('gpu:', torch.cuda.get_device_name(0))

cuda available: True
device: cuda
cuda version: 11.8
gpu: NVIDIA GeForce RTX 3080


### Load data

In [3]:
df = pd.read_csv("../data/smiles_and_targets.csv")
print(np.shape(df))

(132820, 21)


#### Convert data to graphs

In [4]:
from graph_dataset_functions import create_graph_dataset_from_smiles

properties_names = ['A', 'B', 'C', 'mu', 'alfa', 'homo', 'lumo', 'gap', 'R²', 'zpve', 'U0', 'U', 'H', 'G', 'Cv']

x_smiles = df.smiles.values
y = df.loc[:, properties_names].values  # shape = (n_samples, n_properties)

dataset = create_graph_dataset_from_smiles(x_smiles, y[0:len(x_smiles), :])

Information of the graph dataset

In [5]:
print(f'Number of graphs (molecules): {len(dataset)}')
graph = dataset[50]
print('=================================================================================')
print(f'Properties of graph {50}, molecule smiles: {df.smiles.values[50]}')
print(f'Number of nodes: {dataset[50].x.shape[0]}')
print(f'Number of edges: {dataset[50].edge_index.shape[1]}')
print(f'Number of node features: {dataset[50].x.shape[1]}')
print(f'Number of edge features: {dataset[50].edge_attr.shape[1]}')
print(f'Number of target properties: {dataset[50].y.shape[1]}')

Number of graphs (molecules): 132820
Properties of graph 50, molecule smiles: CC1=CNC=C1
Number of nodes: 6
Number of edges: 12
Number of node features: 78
Number of edge features: 10
Number of target properties: 15


Create functions to load the data

In [6]:
def scale_and_split_data(dataset, val_share, test_share):
    # split the dataset into test and validation:
    num_samples = len(dataset)

    num_samples = len(dataset)

    train_indices, val_indices, test_indices = uf.get_data_split_indices(num_samples, val_share=val_share, test_share=test_share)

    train_data = [dataset[i] for i in train_indices]
    val_data = [dataset[i] for i in val_indices]
    test_data = [dataset[i] for i in test_indices]

    # scale the targets
    train_targets = np.concatenate([data.y for data in train_data], axis=0)
    val_targets = np.concatenate([data.y for data in val_data], axis=0)
    test_targets = np.concatenate([data.y for data in test_data], axis=0)    
    
    train_targets, val_targets, test_targets, scaler_targets = uf.scale_targets(train_targets, val_targets, test_targets)

    train_targets = torch.tensor(train_targets, dtype=torch.float, device=device)
    val_targets = torch.tensor(val_targets, dtype=torch.float, device=device)
    test_targets = torch.tensor(test_targets, dtype=torch.float, device=device)


    train_data = [Data(x=data.x.to(device), edge_index=data.edge_index.to(device), edge_attr=data.edge_attr.to(device), 
                       y=train_targets[index].reshape(1,-1)) for index, data in enumerate(train_data)]
    
    val_data = [Data(x=data.x.to(device), edge_index=data.edge_index.to(device), edge_attr=data.edge_attr.to(device),
                        y=val_targets[index].reshape(1,-1)) for index, data in enumerate(val_data)]
    
    test_data = [Data(x=data.x.to(device), edge_index=data.edge_index.to(device), edge_attr=data.edge_attr.to(device), 
                      y=test_targets[index].reshape(1,-1)) for index, data in enumerate(test_data)]

    return train_data, val_data, test_data, scaler_targets

def create_data_loaders(train_data, val_data, test_data, batch_size): 
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)
    
    return train_loader, val_loader, test_loader

Load data

In [7]:
print("...Loading data...")
train_data, val_data, test_data, scaler_targets = scale_and_split_data(dataset, 0.15, 0.1)
train_loader, val_loader, test_loader = create_data_loaders(train_data, val_data, test_data, batch_size=64)
print("...Data loading done...")

...Loading data...
...Data loading done...


## Test GNN Structures

In [8]:
def train(model, batch):
      model.train()
      optimizer.zero_grad()  # Clear gradients.
      out = model(batch.x, batch.edge_index, batch.edge_attr, batch.batch).to(device)  # Perform a single forward pass.

      targets = batch.y
      loss = criterion(out, targets) 

      loss.backward()  # Derive gradients.
      optimizer.step()  # Update parameters based on gradients.
      
      return loss

def test(model, data):
      all_r2 = []
      all_loss = []
      counter = -1    
      for batch in data:
            counter += 1
            model.eval()
            out = model(batch.x, batch.edge_index, batch.edge_attr, batch.batch).cpu()
            targets = batch.y.cpu()
            
            # Caculate R2
            r2_score_var = []
            for item in range(target_dim):
                  if item == 0:
                        r2_score_var = r2_score(targets[:,item].detach().numpy(), out[:,item].detach().numpy())
                  else:
                        new_score = r2_score(targets[:,item].detach().numpy(), out[:,item].detach().numpy())
                        r2_score_var = np.vstack((r2_score_var, new_score))

            if counter == 0:
                  all_r2 = r2_score_var
            else:
                  all_r2 = np.hstack((all_r2, r2_score_var))
                  
            loss = float(criterion(out, targets).detach().numpy())
            all_loss = np.hstack((all_loss, loss))

      average_test_r2 = np.mean(all_r2, axis=1)
      average_test_loss = np.mean(all_loss)
      
      return average_test_r2, average_test_loss  
  
def early_stopping(val_losses, patience): # returns True if there is no improvement in val_loss
      if len(val_losses) < patience:
            return False
      else:
            best_loss = np.min(val_losses)
            current_loss = val_losses[-1]
            
            if current_loss > best_loss:
                  return True
            else:
                  return False

layer_counts = [1, 3, 5, 7]
channel_counts = [256, 512, 1024]
n_epochs = 100

# Run experiment
feature_dim = train_data[0].x.shape[1]
target_dim = train_data[0].y.shape[1]

loss_train_res = np.zeros((len(layer_counts), len(channel_counts)))
loss_val_res = np.zeros((len(layer_counts), len(channel_counts)))
avg_r2_train = np.zeros((len(layer_counts), len(channel_counts)))
avg_r2_val = np.zeros((len(layer_counts), len(channel_counts)))

network_counter = 0
for i, num_layers in enumerate(layer_counts):
    for j, hidden_channels in enumerate(channel_counts):
        network_counter += 1    
        train_start = time.time()
        
        model_class = GNNs.define_GNN_structure(num_layers, hidden_channels, feature_dim, target_dim)
        model = model_class().to(device)
        
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        criterion = torch.nn.MSELoss().to(device)
        
        decay_rate = 0.94
        lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=decay_rate)
        
        print('=================================================================================')
        print(f"Network {network_counter} of {len(layer_counts)*len(channel_counts)}")
        print(f'Number of GAT layers: {num_layers}')
        print(f'Number of hidden channels: {hidden_channels}')
        trainable_params, _ = uf.get_num_parameters(model)
        print(f"Number of trainable parameters: {trainable_params:,}")
        print('=================================================================================')

        val_losses_epoch = []
        epochs_without_improvement = 0
        best_val_loss = np.inf
        patience = 3
        for epoch in np.arange(1, n_epochs+1):
            losses = []
            for batch in train_loader:
                loss = train(model, batch)
                losses.append(loss.cpu().detach().numpy())  
            # Compute validation loss
            model.eval()
            val_losses = []
            for batch in val_loader:          
                targets = batch.y
                out = model(batch.x, batch.edge_index, batch.edge_attr, batch.batch)
                val_losses.append(criterion(out, targets).cpu().detach().numpy()) 
            
            val_loss_epoch = np.mean(val_losses)
            val_losses_epoch.append(val_loss_epoch)
            print(f"Epoch: {epoch:02d} | Train Loss: {np.mean(losses):.5f} | Validation Loss: {val_loss_epoch:.5f}")
            train_end = time.time()
        
            lr_scheduler.step() # Decay to learning rate
            
            # check for early stopping
            if early_stopping(val_losses_epoch, patience) and epoch > patience:
                    epochs_without_improvement += 1
                    if epochs_without_improvement >= patience:
                        print(f"Early stopping at epoch {epoch} after {patience} epochs without improvement.")
                        break
            else:
                    epochs_without_improvement = 0
                    best_val_loss = val_loss_epoch
                    # save the model
                    torch.save(model.state_dict(), 'best_temp_model.pt')
                    
        print("...Training done...")
        print("Time elapsed for training:", (train_end - train_start)/60, " minutes")
        model.load_state_dict(torch.load('best_temp_model.pt'))
        r2_train, loss_train = test(model, train_loader) 
        loss_train_res[i, j] = loss_train
        avg_r2_train[i, j] = np.mean(r2_train)
        
        r2_val, loss_val = test(model, val_loader)
        loss_val_res[i, j] = loss_val
        avg_r2_val[i, j] = np.mean(r2_val)

        print('=================================================================================')
        print("Final training R2:", r2_train)
        print("Average final training R2: ", np.mean(r2_train))
        print("Final training loss:", loss_train)

        print("Final validation R2:", r2_val)
        print("Average final validation R2: ", np.mean(r2_val))
        print("Final validation loss:", loss_val)
print()
print("Experiment done!") 

Network 1 of 12
Number of GAT layers: 1
Number of hidden channels: 256
Number of trainable parameters: 122,895
Epoch: 01 | Train Loss: 0.33341 | Validation Loss: 0.28104
Epoch: 02 | Train Loss: 0.27640 | Validation Loss: 0.25188
Epoch: 03 | Train Loss: 0.25518 | Validation Loss: 0.23293
Epoch: 04 | Train Loss: 0.24089 | Validation Loss: 0.22403
Epoch: 05 | Train Loss: 0.23007 | Validation Loss: 0.21362
Epoch: 06 | Train Loss: 0.22260 | Validation Loss: 0.20694
Epoch: 07 | Train Loss: 0.21422 | Validation Loss: 0.20300
Epoch: 08 | Train Loss: 0.20776 | Validation Loss: 0.18928
Epoch: 09 | Train Loss: 0.20094 | Validation Loss: 0.19206
Epoch: 10 | Train Loss: 0.19568 | Validation Loss: 0.17617
Epoch: 11 | Train Loss: 0.19083 | Validation Loss: 0.16943
Epoch: 12 | Train Loss: 0.18590 | Validation Loss: 0.16666
Epoch: 13 | Train Loss: 0.18068 | Validation Loss: 0.15737
Epoch: 14 | Train Loss: 0.17761 | Validation Loss: 0.15324
Epoch: 15 | Train Loss: 0.17400 | Validation Loss: 0.14877
Epoc

In [10]:
print("...Saving results...")
df_loss_train = pd.DataFrame(loss_train_res, index=layer_counts, columns=channel_counts)
df_loss_val = pd.DataFrame(loss_val_res, index=layer_counts, columns=channel_counts)
df_avg_r2_train = pd.DataFrame(avg_r2_train, index=layer_counts, columns=channel_counts)
df_avg_r2_val = pd.DataFrame(avg_r2_val, index=layer_counts, columns=channel_counts)

df_loss_train.to_csv('experiment_results/loss_train.csv')
df_loss_val.to_csv('experiment_results/loss_val.csv')
df_avg_r2_train.to_csv('experiment_results/avg_r2_train.csv')
df_avg_r2_val.to_csv('experiment_results/avg_r2_val.csv')

...Saving results...


In [11]:
print("Training loss:")
print(df_loss_train)

Training loss:
       256       512       1024
1  0.109000  0.114687  0.092053
3  0.061234  0.051048  0.037577
5  0.058765  0.047773  0.040866
7  0.069952  0.044502  0.046594


In [12]:
print("Average training R2:")
print(df_avg_r2_train)

Average training R2:
       256       512       1024
1  0.886772  0.880900  0.903675
3  0.935892  0.946446  0.960578
5  0.938245  0.949765  0.957036
7  0.926317  0.952965  0.950969


In [13]:
print("Validation loss:")
print(df_loss_val)

Test loss:
       256       512       1024
1  0.113837  0.119683  0.097134
3  0.068668  0.059405  0.048517
5  0.067369  0.057372  0.049545
7  0.077003  0.056084  0.056143


In [14]:
print("Average validation R2:")
print(df_avg_r2_val)

Average test R2:
       256       512       1024
1  0.881948  0.876203  0.898806
3  0.927869  0.937633  0.949127
5  0.929266  0.939805  0.948007
7  0.919293  0.941216  0.940847


### Graph Neural Network

#### Model for all targets at once

Train GNN

In [25]:
def train(model, batch):
      model.train()
      optimizer.zero_grad()  # Clear gradients.
      out = model(batch.x, batch.edge_index, batch.edge_attr, batch.batch).to(device)  # Perform a single forward pass.

      targets = batch.y
      loss = criterion(out, targets) 

      loss.backward()  # Derive gradients.
      optimizer.step()  # Update parameters based on gradients.
      return loss

def test(model, batch):
      all_r2 = []
      all_loss = []
      counter = -1    
      for i, batch in enumerate(batch):
            counter += 1
            model.eval()
            out = model(batch.x, batch.edge_index, batch.edge_attr, batch.batch).cpu()
            targets = batch.y.cpu()
            
            # Caculate R2 for each target
            for target_idx in range(target_dim):
                  if target_idx != 0:
                        r2_score_var = np.vstack((r2_score_var, r2_score(targets[:,target_idx].detach().numpy(), 
                                                          out[:,target_idx].detach().numpy())))
                  else:
                        r2_score_var = np.array([r2_score(targets[:,target_idx].detach().numpy(),
                                                          out[:,target_idx].detach().numpy())])        
            all_r2 = np.hstack((all_r2, r2_score_var)) if i != 0 else r2_score_var
                  
            loss = float(criterion(out, targets).detach().numpy())
            all_loss = np.hstack((all_loss, loss)) if i != 0 else np.array(loss)

      average_test_r2 = np.mean(all_r2, axis=1)
      average_test_loss = np.mean(all_loss)
      
      return average_test_r2, average_test_loss

def early_stopping(val_losses, patience): # returns True if there is no improvement in val_loss
      if len(val_losses) < patience:
            return False
      else:
            best_loss = np.min(val_losses)
            current_loss = val_losses[-1]
            
            if current_loss > best_loss:
                  return True
            else:
                  return False

feature_dim = train_data[0].x.shape[1]
target_dim = train_data[0].y.shape[1]

num_layers, hidden_channels = 5, 1024
model_class = GNNs.define_GNN_structure(num_layers, hidden_channels, feature_dim, target_dim)
model = model_class().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
criterion = torch.nn.MSELoss().to(device)

decay_rate = 0.94
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=decay_rate)

train_params, _ = uf.get_num_parameters(model)
print(f"Trainable parameters: {train_params:,}")
      
# Vectors to append accuracy to:
train_r2 = []
train_loss = []
test_r2 = []
test_loss = []
val_r2 = []
val_loss = []

n_epochs = 100
print_every_N_epochs = False
N = 10 # print R2 every N epochs

epoch_times = []
train_times = []
print()
print("...Starting training...")
print("Device used:", device)

val_losses_epoch = [] # for early stopping
patience = 5 # how many epochs to wait for the val loss to improve
best_val_loss = np.inf  
epochs_without_improvement = 0

for epoch in np.arange(1, n_epochs+1):
      epoch_start = time.time()
      losses = []
      train_start = time.time()
      for batch in train_loader:
            loss = train(model, batch)
            losses.append(loss.cpu().detach().numpy())  
      # Compute validation loss
      model.eval()
      val_losses = []
      for batch in test_loader:
            targets = batch.y
            out = model(batch.x, batch.edge_index, batch.edge_attr, batch.batch)
            val_losses.append(criterion(out, targets).cpu().detach().numpy())
      
      val_loss_epoch = np.mean(val_losses)
      val_losses_epoch.append(val_loss_epoch)
      print(f"Epoch: {epoch:02d} | Train Loss: {np.mean(losses):.5f} | Validation Loss: {val_loss_epoch:.5f}")
      train_end = time.time()
      train_times.append(train_end - train_start)
      lr_scheduler.step() # Decay to learning rate
      
      # check for early stopping
      if early_stopping(val_losses_epoch, patience) and epoch > patience:
            epochs_without_improvement += 1
            if epochs_without_improvement >= patience:
                  print(f"Early stopping at epoch {epoch} after {patience} epochs without improvement.")
                  break
      else:
            epochs_without_improvement = 0
            best_val_loss = val_loss_epoch
            # save the model
            torch.save(model.state_dict(), 'best_graphs_only_model.pt')
                   
      if print_every_N_epochs and (epoch % N == 0 or epoch == 1) and epoch != n_epochs:
            test_start = time.time()
                
            r2_temp_val, loss_temp_val = test(model, test_loader)
            val_r2.append(r2_temp_val)
            val_loss.append(loss_temp_val)
            
            print(f"Validation loss: {loss_temp_val}")
            print(f'Validation R2: {r2_temp_val}')
            print(f"Average validation R2: {np.mean(r2_temp_val):.5f}")
            
      epoch_end = time.time()
      epoch_times.append(epoch_end - epoch_start)
      
print("...Training done...")
print("...Calculating final results...")
model.load_state_dict(torch.load('best_graphs_only_model.pt'))

r2_temp_train, loss_temp_train = test(model, train_loader) 
train_r2.append(r2_temp_train)
train_loss.append(loss_temp_train)

r2_temp_val, loss_temp_val = test(model, test_loader)
val_r2.append(r2_temp_val)
val_loss.append(loss_temp_val)

r2_temp_test, loss_temp_test = test(model, val_loader)
test_r2.append(r2_temp_test)
test_loss.append(loss_temp_test)

print("====================================================")
print("Final training R2:", train_r2[-1])
print("Average final training R2: ", np.mean(train_r2[-1]))
print("Final training loss:", train_loss[-1])

print("Final validation R2:", val_r2[-1])
print("Average validation R2: ", np.mean(val_r2[-1]))
print("Final validation loss:", val_loss[-1])

print("Final test R2:", test_r2[-1])
print("Average final test R2: ", np.mean(test_r2[-1]))
print("Final test loss:", test_loss[-1])

Trainable parameters: 5,489,935

...Starting training...
Device used: cuda
Epoch: 01 | Train Loss: 0.30232 | Validation Loss: 0.25115
Epoch: 02 | Train Loss: 0.23202 | Validation Loss: 0.21343
Epoch: 03 | Train Loss: 0.20464 | Validation Loss: 0.18316
Epoch: 04 | Train Loss: 0.18135 | Validation Loss: 0.16385
Epoch: 05 | Train Loss: 0.16348 | Validation Loss: 0.14297
Epoch: 06 | Train Loss: 0.14979 | Validation Loss: 0.13316
Epoch: 07 | Train Loss: 0.13735 | Validation Loss: 0.12096
Epoch: 08 | Train Loss: 0.12913 | Validation Loss: 0.11523
Epoch: 09 | Train Loss: 0.12115 | Validation Loss: 0.10875
Epoch: 10 | Train Loss: 0.11224 | Validation Loss: 0.09823
Epoch: 11 | Train Loss: 0.10771 | Validation Loss: 0.09409
Epoch: 12 | Train Loss: 0.10207 | Validation Loss: 0.08912
Epoch: 13 | Train Loss: 0.09671 | Validation Loss: 0.08178
Epoch: 14 | Train Loss: 0.09317 | Validation Loss: 0.08296
Epoch: 15 | Train Loss: 0.08834 | Validation Loss: 0.07752
Epoch: 16 | Train Loss: 0.08534 | Valida

In [ ]:
print("Device used:", device)
print()
print(f"Total number of epochs: {len(epoch_times)}")
print(f"Total training time: {np.sum(epoch_times)/60:.2f} minutes")
print(f"Total time in training: {np.sum(train_times)/60:.2f} minutes")
print()
print(f"Average epoch time: {np.mean(epoch_times):.1f} seconds")
print(f"Average time in training: {np.mean(train_times):.1f} seconds")

NameError: name 'device' is not defined

#### Model for just one target

GNN function

Train GNN

In [ ]:
feature_dim = train_data[0].x.shape[1]
target_dim = 1

def train(data_in, target):
      model.train()
      optimizer.zero_grad()  # Clear gradients.
      out = model(data_in.x, data_in.edge_index, data_in.edge_attr, data_in.batch).to(device)
      targets = data_in.y[:,target].reshape(-1,1)
      
      loss = criterion(out, targets)   

      loss.backward()  # Derive gradients.
      optimizer.step()  # Update parameters based on gradients.
      return loss

def test(data, target):
      all_r2 = []
      all_loss = []
      counter = -1    
      for data_in in data:
            counter += 1
            model.eval()
            out = model(data_in.x, data_in.edge_index, data_in.edge_attr, data_in.batch).cpu()
            targets = data_in.y[:,target].cpu().reshape(-1,1)
            
            # Caculate R2
            r2_score_var = r2_score(targets.detach().numpy(), out.detach().numpy())
            all_r2.append(r2_score_var)
            
            loss = float(criterion(out, targets).detach().numpy())
            all_loss.append(loss)

      average_test_r2 = np.mean(all_r2)
      average_test_loss = np.mean(all_loss)

      return average_test_r2, average_test_loss

num_targets = train_data[0].y.shape[1]
start_time = time.time()
for target_index in range(num_targets):
      print("Target index:", target_index)

      model = GNN(hidden_channels=64, feature_dim=feature_dim, target_dim=target_dim).to(device) 
      optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=5e-4)
      criterion = torch.nn.MSELoss().to(device)

      # Vectors to append accuracy to:
      train_r2 = []
      test_r2 = []
      train_loss = []
      test_loss = []

      # Calculate accuracy and loss before training 
      r2_temp, loss_temp = test(train_loader, target_index)
      train_r2.append(r2_temp)
      train_loss.append(loss_temp)
      r2_temp, loss_temp = test(test_loader, target_index)
      test_r2.append(r2_temp)
      test_loss.append(loss_temp)
      
      print("Initial training R2: ", train_r2[0])
      print("Initial test R2: ", test_r2[0])

      print_r2_option = True
      counter = 0
      for epoch in range(1, 21):
            counter += 1
            losses = []
            for data in train_loader:
                  loss = train(data, target_index)
                  losses.append(loss.cpu().detach().numpy())
            print(f'Epoch: {epoch:03d}, Loss: {np.mean(losses):.5f}')

            if print_r2_option & epoch == 20:
                  temp_train_r2, temp_train_loss = test(train_loader, target_index)
                  train_r2.append(temp_train_r2)
                  train_loss.append(temp_train_loss)

                  temp_test_r2, temp_test_loss = test(test_loader, target_index)
                  test_r2.append(temp_test_r2)
                  test_loss.append(temp_test_loss)

      print(f"Best training R2 for target {target_index}: {np.max(train_r2)}")
      print(f"Best test R2 for target {target_index}: {np.max(test_r2)}")
print("...Done...")
end_time = time.time()
print(f"Time taken: {(end_time - start_time)/60} minutes")
print(f"Average time per target: {(end_time - start_time)/(num_targets*60)} minutes")

Target index: 0
Initial training R2:  -0.006660032108331978
Initial test R2:  -0.00475941961664909
Epoch: 001, Loss: 0.68831


KeyboardInterrupt: 